# SVD

In [5]:
import numpy as np

A = np.array([[1.,2.,0.],[2.,1.,0.],[0.,0.,1.]])
U = np.array([[1/(np.sqrt(2)), 0, -1/np.sqrt(2)], [1/np.sqrt(2), 0, 1/np.sqrt(2)], [0, 1, 0]])
V = np.array([[1/np.sqrt(2), 0, 1/np.sqrt(2)], [1/np.sqrt(2), 0, -1/np.sqrt(2)], [0,1,0]])
Σ = np.array([[3,0,0], [0,1,0],[0,0,1]])

print(A - U@Σ@np.transpose(V))

[[1.11022302e-16 2.22044605e-16 0.00000000e+00]
 [2.22044605e-16 1.11022302e-16 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]


# Romberg

In [ ]:
from numpy import exp
from copy import deepcopy

def equiX(start, end, m):
    stepSize = (end-start)/m
    return [start+i*stepSize for i in range(m+1)]


def compTrap(f, start, end, m):
    h = (end-start)/m
    xs = equiX(start, end, m)
    midPoints = sum([f(xs[i]) for i in range(1, m)])
    return h*(f(xs[0])/2+midPoints+f(xs[-1])/2)

def romberg(f, start, end, maximum):
    zeros = [0 for i in range(maximum)]
    initial = [compTrap(f, start, end, 2**m) for m in range(1,maximum+1)]
    algoMatrix = [initial if i == 0 else deepcopy(zeros) for i in range(maximum)]
    for k in range(1, maximum):
        for i in range(k, maximum):
            algoMatrix[k][i] = ((4**k)*algoMatrix[k-1][i]-algoMatrix[k-1][i-1])/((4**k)-1)
    return algoMatrix


def f(x):
    return exp(-1*(x**2))
print(romberg(f, 0, 1, 4)[2][3])

# Lotka-Volterra

In [ ]:
import matplotlib.pyplot as plt
import scipy.integrate as ode
from math import sqrt

def equiX(start, end, m):
    stepSize = (end-start)/m
    return [start+i*stepSize for i in range(m+1)]

# y' = F(x, y), hvor y : R -> R, F : RxR^n -> R^n
def euler(F, start, end, steps, y0):
    h = (end-start)/steps
    xs = equiX(start, end, steps)
    ys = [y0]
    for x in xs[1:]:
        temp = []
        yn = ys[-1]
        f = F(x, yn)
        for i in range(len(yn)):
            temp.append(yn[i] + h*f[i])
        ys.append(temp)
    return ys

# y' = F(x, y), hvor y : R -> R, F : RxR^n -> R^n
def heun(F, start, end, steps, y0):
    h = (end-start)/steps
    xs = equiX(start, end, steps)
    ys = [y0]
    for x in xs[1:]:
        temp = []
        eul = []
        yn = ys[-1]
        f = F(x, yn)
        for i in range(len(yn)):
            eul.append(yn[i] + h*f[i])
        for i in range(len(yn)):
            g = F(x+h, eul)
            temp.append(yn[i] + h/2*(f[i] + g[i]))
        ys.append(temp)
    return ys

def err2(xs, ys, start, end, steps):
    return (end-start)/steps*sqrt(sum([(y-x)**2 for (x,y) in zip(xs, ys)]))

ls = equiX(0,10,1000)
α, β, δ, γ = 4, 1.2, 0.4, 1.6
F = lambda x, ys: [α*ys[0]-β*ys[0]*ys[1], δ*ys[0]*ys[1]-γ*ys[1]]
init = [10, 10]
# eTest1 = euler(F, 0, 100, 10000, init)
# eTest2 = euler(F, 0, 100, 100000, init)
# eTest3 = euler(F, 0, 100, 1000000, init)
# hTest = heun(F, 0, 100, 10000, init)
# bestTest = list(map(ode.solve_ivp(G, [0, 100], init, dense_output = True).sol, ls))

#Euler error
def EulerError(sys, ini, start, end, iterable):
    errls = []
    superIntendent = ode.solve_ivp(sys, [start, end], ini, dense_output = True)
    for i in iterable:
        ls = equiX(start, end, i)
        test = euler(sys, start, end, i, ini)
        comp = list(map(superIntendent.sol, ls))
        err = 0
        for j in range(len(test[0])):
            err += err2([t[j] for t in test], [c[j] for c in comp], start, end, i)
        errls.append(err)
    return errls

#Heun Error
def HeunError(sys, ini, start, end, iterable):
    errls = []
    superIntendent = ode.solve_ivp(sys, [start, end], ini, dense_output = True)
    for i in iterable:
        ls = equiX(start, end, i)
        test = heun(sys, start, end, i, ini)
        comp = list(map(superIntendent.sol, ls))
        err = 0
        for j in range(len(test)):
            err += err2(test[j], comp[j], start, end, i)
        errls.append(err)
    return errls

# plt.figure()
# plt.axes(xlabel = "Prey", ylabel = "Predator")
# plt.title("Lotka-Volterra System")
# # plt.plot([v[0] for v in eTest1], [v[1] for v  in eTest1], 'c')
# plt.plot([v[0] for v in eTest2], [v[1] for v  in eTest2], 'b', label = "Coarse")
# plt.plot([v[0] for v in eTest3], [v[1] for v  in eTest3], 'k', label = "Fine")
# plt.plot([v[0] for v in hTest], [v[1] for v in hTest], 'r', label = "Heun")
# plt.plot([v[0] for v in bestTest], [v[1] for v in bestTest], 'm', label = "Sci")
# plt.legend()
# plt.show()

errsEul = EulerError(F, init, 0, 10, [i*1000 for i in range(1, 101)])
errsHeun = HeunError(F, init, 0, 1, [i*1000 for i in range(1, 101)])
plotls = [1/(i*1000) for i in range(1, 101)]
errsEul.reverse()
errsHeun.reverse()
plotls.reverse()
plt.figure()
plt.axes(xlabel = "h", ylabel = "error")
plt.title(f"Convergence of Methods w/ α={α}, β={β}, δ={δ} and γ={γ}")
plt.loglog(plotls, errsEul, label = "Euler")
plt.loglog(plotls, errsHeun, label = "Heun")
plt.legend()
plt.show()

# Pendulum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def equiX(start, end, m):
    stepSize = (end-start)/m
    return np.array([start+i*stepSize for i in range(m+1)])

def Ah(dim):
    one = np.array([1 for i in range(dim-1)])
    two = np.array([-2 for i in range(dim)])
    return np.diag(two) + np.diag(one, -1) + np.diag(one, 1)


def bvp(ω, F, start, end, α, β, M):
    h = (end-start)/(M+1)
    def choice(i):
        if i == 1:
            return α/(h**2)
        elif i == M:
            return β/(h**2)
        else:
            return 0

    xs = equiX(start, end, M+1)
    Gh = (1/h**2)*Ah(M) + ω**2*np.identity(M)
    print(Gh)
    b = np.array([F[i-1](xs[i]) + choice(i) for i in range(1,M+1)])
    return np.linalg.solve(Gh, b)

plt.figure()
plt.plot([i/1000 for i in range(1, 1001)], bvp(1,[lambda x: 0 for i in range(1000)], 0, 1, 0, 1, 1000))
plt.show()